## << pretreatment >>

### 1. Integration of data (when necessary)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# File path setting
base_path = '/content/drive/MyDrive/Kwargs/011. 뉴스 크롤링 결과/csv/'

#File name format setting
file_format = '_news_data.csv'

# A list to integrate data from all companies
all_data = []

# Check all the files in the file path and read the file that fits the format
file_list = [file_name for file_name in os.listdir(base_path) if file_name.endswith(file_format)]

# Use TQDM to show the progress of the file reading process.
for file_name in tqdm(file_list, desc="Reading files"):
# Extracting 'Company Name' (Remove '_news_data.csv' in the file name)
    company_name = file_name.replace(file_format, '')

# Read file
    file_path = os.path.join(base_path, file_name)
    df = pd.read_csv(file_path)

# Add company column
    df['company'] = company_name

# Add to data frame list
    all_data.append(df)

# Integrate all data
if all_data:
# Use TQDM to show the progress of the data integration process.
    combined_df = pd.concat(tqdm(all_data, desc="Combining data"), ignore_index=True)

# Change value of Company column
    combined_df['company'] = combined_df['company'].replace('포스코 홀딩스', 'POSCO홀딩스')
    combined_df['company'] = combined_df['company'].replace('GS 에너지', 'GS에너지')

#Save integrated DataFrame as a file
    combined_df.to_csv('/content/drive/MyDrive/Kwargs/011. 뉴스 크롤링 결과/A0_combined.csv', index=False)

    print("모든 기업 데이터를 통합했고, 'company' 컬럼을 추가했습니다.")
else:
    print("통합할 데이터가 없습니다.")


Combining data: 100%|██████████| 69/69 [00:00<00:00, 186113.81it/s]


모든 기업 데이터를 통합했고, 'company' 컬럼을 추가했습니다.


In [ ]:
combined_df.head()

,title,date,content,category,company
0,“덜 준 보험금 달라” 2000명 무더기 신청…보험사는 ‘좌불안석’,2019-01-15 06:01:01,- 금감원에 ''즉시연금'' 민원 몰려- 70%가 삼성·한화·교보생명에 집중- 소멸...,경제,삼성생명
1,KEB하나은행 신지현 '빠른 드리블',2019-02-18 19:24:58,(부천뉴스1) 정진욱 기자 18일 오후 경기도 부천체육관에서 열린 ‘2018-20...,사회,삼성생명
2,ET투자뉴스]LG_관련종목들 중에서 변동성은 낮지만 수익률은 높아,2019-01-15 14:30:01,관련종목들 중에서 변동성은 낮지만 수익률은 높아 15일 오후 2시20분 현재 전일...,경제,삼성생명
3,이동걸식 대우조선해양 해법은 '국민혈세 몰아주기'?,2019-02-07 09:22:15,한국GM에 가성비 논리를 펴가며 8000여억원의 자금을 지원한 이동걸 KDB산업은행...,경제,삼성생명
4,ET투자뉴스]SK이노베이_상장주식수 대비 거래량은 0.3%로 적정수준,2019-01-09 13:05:41,"위험대비수익률 -2.4, 업종평균대비 성과 좋아 9일 오후 12시20분 현재 전일...",경제,삼성생명


In [ ]:
combined_df.shape

(1383707, 5)

### 2. Text clustering to remove sports, subcommittee, stock chirashi (planned)

https://colab.Research.google.com/drive/1bcWkxg8zbccxi7mmqcilex2wchbeor#sscrollto=EKT3-UDEL1ZD

In [ ]:
### You have to write. # https://chatgpt.com/c/66f6f0d0-4c84-8006-bfab-99018660CF34
#Un


### 3. Text pretreatment

In [ ]:
#df = combined_df.copy()
df = pd.read_csv('/content/drive/MyDrive/Kwargs/011. 뉴스 크롤링 결과/A0_combined.csv')

In [ ]:
from tqdm import tqdm
import pandas as pd
import re

# The last anti -processing function by media company
def process_article_footer(text):
# Definition of patterns by media company
    patterns = {
        "서울경제": [r'저작권자\s서울경제.*?전재\s및\s재배포\s금지', r'@sedaily\.com'],
        "파이낸셜뉴스": [r'저작권자\s파이낸셜뉴스.*?무단\s전재-재배포\s금지', r'@fnnews\.com'],
        "쿠키뉴스": [r'무단전재\s및\s재배포금지', r'@kukinews\.com'],
        "이데일리": [r'＜종합\s경제정보\s미디어\s이데일리\s-\s무단전재\s재배포\s금지＞', r'@edaily\.co\.kr'],
        "중앙일보": [r'중앙일보.*?무단\s전재\s및\s재배포\s금지', r'@joongang\.co\.kr'],
        "조선비즈": [r'저작권자\s조선비즈.*?무단\s전재\s및\s재배포\s금지', r'@chosunbiz\.com'],
        "데일리안": [r'데일리안\s-\s무단전재,\s변형,\s무단배포\s금지', r'@dailian\.co\.kr'],
        "머니투데이": [r'머니투데이.*?기자', r'@mt\.co\.kr'],
        "더팩트": [r'저작권자\s특종에\s강한\s더팩트.*?재배포\s금지', r'@tf\.co\.kr'],
        "매일경제": [r'저작권자\s매일경제.*?mk\.co\.kr', r'@mk\.co\.kr'],
        "연합뉴스": [r'연합뉴스TV.*?제보', r'@yna\.co\.kr'],
        "한국경제": [r'저작권자\s한국경제.*?', r'@hankyung\.com'],
        "아시아경제": [r'@asiae\.co\.kr', r'아시아경제'],
        "전자신문": [r'전자신문.*?전자신문인터넷', r'@etnews\.com'],
        "CBS노컷뉴스": [r'©\s뉴스1코리아.*?무단\s전재\s및\s재배포\s금지', r'@cbs\.co\.kr'],
        "아이뉴스24": [r'아이뉴스24.*?', r'@inews24\.com']
    }

    for agency, agency_patterns in patterns.items():
        for pattern in agency_patterns:
            if len(text) > 200:
                last_200_chars = text[-200:]
                last_200_chars = re.sub(pattern, '', last_200_chars)
                text = text[:-200] + last_200_chars
            else:
                text = re.sub(pattern, '', text)

    return text

#Protocation function related to the reporter's name
def remove_trailing_journalist_name(text):
    if not text.endswith('.'):
        last_chars = text[-4:] if len(text) >= 4 else ''

        if last_chars:
            if last_chars[0] in ['.', ' ']:
                text = text[:-4] + last_chars[0]
            elif last_chars[0] == '다':
                text = text[:-4] + "다"
    return text

# Pre -treatment function (including all text deletion after certain phrases)
def preprocess_article(text):
    special_patterns = [
        r'CBS노컷뉴스는\s여러분의\s제보로',
        r'유튜브와\s포털에서\s각각',
        r'저작권자\.\s*c\.\s*오마이뉴스',
        r'본\s기사는',
        r'특별취재팀',
        r'https?:\/\/[^\s]+',
        r'기자\s?[A-Za-z]+\s?[A-Za-z]*@[^\s]+',
        r'[가-힣]+\s기자\s?[가-힣]+\s?[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}',
        r'[가-힣]+\s기자',
        r'(?:\s|^)' + r'발로 뛰는 더팩트는',
        r'네이버 홈에서',
        r'＜종합 경제정보',
        r'세상을 보는 눈,',
        r'당신의 제보가',
        r'WEEKLY BIZ Newsletter',
        r'제보하기 :'
    ]

    for pattern in special_patterns:
        text = safe_remove_pattern(text, pattern)

# All text removal after email
    text = re.sub(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}.*(기자|무단\s전재|재배포\s금지).*', '', text, flags=re.IGNORECASE)

#Remove the reporter's name and pattern
    text = re.sub(r'\b[가-힣]{2,4}기자\b|\b[가-힣]{2,4} 기자\b', '', text)

# Remove the later promotional phrase after removing the email in parentheses
    text = re.sub(r'\([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}.*?\)', '', text)

#Remove promotional phrases
    text = re.sub(r'네이버에서.*?뉴스 확인', '', text)
    text = re.sub(r'연합뉴스TV.*?입니다.', '', text)

#Metter information removal
    meta_patterns = [
        r'저작권자\s.+?\s무단\s전재\s및\s재배포\s금지',
        r'무단\s전재-재배포\s금지',
        r'GoodNews paper',
        r'제보\s이메일',
        r'카카오톡',
        r'네이버\s홈에서\s‘.+?’\s뉴스',
        r'\(사진연합뉴스\)',
        r'종합\s경제정보\s미디어\s-\s무단전재\s재배포\s금지',
        r'뉴스\s홈페이지',
        r'기자문의\s및\s제보'
    ]

    for pattern in meta_patterns:
        text = re.sub(pattern, '', text)

#Great theorem
    text = re.sub(r'\s+', ' ', text).strip()

# Last half -handling by media company
    text = process_article_footer(text)

#Check that the reporter's name remains, remove it
    text = remove_trailing_journalist_name(text)

# Divide into a sentence unit and leave only meaningful sentences
    sentences = text.split('. ')
    filtered_sentences = [sentence for sentence in sentences if len(sentence.strip()) > 10 or sentence.isupper()]

# Combined with text again
    return '. '.join(filtered_sentences)

# Safe pattern removal function (only within the last 200 characters)
def safe_remove_pattern(text, pattern):
    if len(text) > 200:
        last_200_chars = text[-200:]
        last_200_chars = re.sub(pattern + r'.*', '', last_200_chars)
        text = text[:-200] + last_200_chars
    return text

tqdm.pandas()
df['content'] = df['content'].progress_apply(preprocess_article)

# Add'Cleaned_Content 'column to the original DF
processed_df = df.copy()

#Save as Processed_df
processed_df.to_csv('/content/drive/MyDrive/Kwargs/011. 뉴스 크롤링 결과/A0_combined_preprocessed.csv', index=False)
processed_df.sample()

100%|██████████| 1383707/1383707 [06:22<00:00, 3621.51it/s]


,title,date,content,category,company
220736,"'미래 탄소 가격까지 따진다'···SK이노베이션, 넷제로 리더십 강화",2022-11-27 10:05:59,"SK이노베이션이 미래 탄소 가격 변화를 반영해 투자 안건 심의에 적용, 탄소 배출량...",경제,sk이노베이션


### 4. Save divided by batch

In [ ]:
import pandas as pd
from tqdm import tqdm

# Bring dataset
file_path = '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_preprocessed.csv'
df = pd.read_csv(file_path)

# Setting size to be divided (200,000 or 100,000)
batch_size = 9999  # 혹은 100000으로 설정 가능

# Division dividing and storage
total_batches = len(df) // batch_size + 1  # 나누어진 배치 개수 계산

for i in tqdm(range(total_batches), desc="데이터셋 나누기"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    batch_df = df.iloc[start_idx:end_idx]
    batch_df.to_csv(f'/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_batch/A0-60_batch_{i}.csv', index=False)
    print(f'Batch {i} 저장 완료: {start_idx} ~ {end_idx}행')

print(f'전체 {total_batches}개의 배치로 나누어 저장 완료되었습니다.')


데이터셋 나누기:   1%|          | 1/139 [00:03<07:54,  3.44s/it]

Batch 0 저장 완료: 0 ~ 9999행


데이터셋 나누기:   1%|▏         | 2/139 [00:04<04:19,  1.89s/it]

Batch 1 저장 완료: 9999 ~ 19998행


데이터셋 나누기:   2%|▏         | 3/139 [00:05<03:11,  1.41s/it]

...

Batch 136 저장 완료: 1359864 ~ 1369863행


데이터셋 나누기:  99%|█████████▉| 138/139 [01:44<00:00,  1.79it/s]

Batch 137 저장 완료: 1369863 ~ 1379862행


데이터셋 나누기: 100%|██████████| 139/139 [01:44<00:00,  1.33it/s]

Batch 138 저장 완료: 1379862 ~ 1383707행
전체 139개의 배치로 나누어 저장 완료되었습니다.


### 5. Treatment of unification mapping & anonymously

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import re
import random
import unicodedata

In [ ]:
#Load Company Names from CSV if needed
company_names_df = pd.read_csv('/content/drive/MyDrive/Kwargs/companynames.csv',encoding='euc-kr')
company_names_df = company_names_df.dropna()
all_companies = company_names_df['종목명'].tolist()

In [ ]:
# Company name unification mapping (existing dictionary)
company_name_mapping = {
    '삼성전자': ['Samsung Electronics', '삼성전자', 'SAMSUNG', '삼성', '삼전'],
    'POSCO홀딩스': ['POSCO홀딩스', '포스코홀딩스', '포스코그룹', 'POSCO', '포스코'],
    '현대백화점': ['Hyundai Department Store', '현대백화점', 'HDC', '현백'],
    'SK이노베이션': ['에스케이이노베이션', 'SK Innovation', 'SK이노베이션', 'SKI'],
    '대한항공': ['Korean Air', '대한항공', 'KAL'],
    '삼성물산': ['Samsung C&T', '삼성물산', '삼성건설',],
    '현대차': ['Hyundai Motor', '현대자동차', 'Hyundai', '현대차', 'HMC'],
    '금호석유': ['금호석유화학', 'Kumho Petrochemical', '금호석유', '금호화학', '금호석화'],
    '카카오': ['다음카카오', 'KAKAO', 'Kakao', '카카오'],
    '효성': ['Hyosung', '효성그룹', '효성'],
    '동국제강': ['Dongkuk Steel', 'DONGKUK', '동국제강'],
    '하림지주': ['Harim Holdings', '하림지주', '하림그룹', '하림'],
    'CJ제일제당': ['CJ CheilJedang', 'CJ제일제당', 'CJ그룹', 'CJ', '씨제이제일제당','씨제이', '제일제당'],
    'LG에너지솔루션': ['LG Energy Solution', 'LG에너지솔루션', 'LG에솔', 'LGES'],
    '삼성바이오로직스': ['Samsung Biologics', '삼성바이오로직스', '삼성바이오','삼바'],
    '셀트리온': ['CELLTRION', 'Celltrion', '셀트리온'],
    '하나금융지주': ['Hana Financial Group', '하나금융지주', '하나금융', '하나은행'],
    'NAVER': ['네이버주식회사', 'NAVER', '네이버'],
    'LG전자': ['LG Electronics', 'LG전자', 'LGE', '엘지전자'],
    '한화에어로스페이스': ['Hanwha Aerospace', '한화에어로스페이스', '한화에어로'],
    'GS에너지': ['GS Energy', 'GS에너지', 'GS 에너지'],
    '우리금융지주': ['Woori Financial Group', '우리금융지주', '우리금융', '우리은행'],
    '카카오뱅크': ['KakaoBank', '카카오뱅크', '카뱅'],
    'KB금융': ['KB Kookmin Bank', 'KB금융지주', 'KB금융', '국민은행'],
    '한화': ['Hanwha', '한화그룹', '한화지주', '한화'],
    '삼성중공업': ['Samsung Heavy Industries', '삼성중공업', 'SHI'],
    '기아': ['기아자동차', 'Kia Motors', 'Kia', '기아차', '기아'],
    '크래프톤': ['KRAFTON', 'Krafton', '크래프톤'],
    'LG화학': ['엘지화학', 'LG Chem', 'LG화학'],
    '삼성SDS': ['삼성에스디에스', 'Samsung SDS', '삼성SDS'],
    'SK텔레콤': ['SK Telecom', 'SK텔레콤','에스케이텔레콤', 'SKT'],
    'SK바이오팜': ['SK Biopharmaceuticals', 'SK BioPharm', 'SK바이오팜'],
    '아모레퍼시픽': ['Amorepacific', '아모레퍼시픽', '아모레'],
    '한국타이어앤테크놀로지': ['한국타이어앤테크놀로지', 'Hankook Tire', '한국타이어'],
    'LG생활건강': ['엘지생활건강', 'LG생활건강', 'LG H&H'],
    'SK하이닉스': ['에스케이하이닉스','SK하이닉스', 'SK hynix', '하이닉스'],
    '한온시스템': ['Hanon Systems', '한온시스템', '한온'],
    'LG유플러스': ['LG유플러스', 'LG U+', '엘지유플러스'],
    '현대모비스': ['Hyundai Mobis', '현대모비스', '모비스'],
    'LG디스플레이': ['LG Display', '엘지디스플레이','LG디스플레이', 'LGD'],
    '롯데지주': ['Lotte Corporation', '롯데지주'],
    'GS리테일': ['GS Retail', 'GS리테일', 'GS25' ],
    '넷마블': ['Netmarble', '넷마블'],
    'SK바이오사이언스': ['SK Bioscience', 'SK바이오사이언스', 'SK바사'],
    '롯데쇼핑': ['Lotte Shopping', '롯데쇼핑'],
    'GS': ['GS그룹', 'GS지주' 'GS'],
    'BGF리테일': ['BGF Retail', 'BGF리테일', 'BGF'],
    '한미약품': ['Hanmi Pharmaceutical', '한미약품'],
    'SK네트웍스': ['SK Networks', 'SK네트웍스'],
    '삼성전기': ['Samsung Electro-Mechanics', '삼성전기'],
    'SK아이이테크놀로지': ['SK IE Technology', 'SK아이이테크놀로지', 'SKIET'],
    'HMM': ['Hyundai Merchant Marine', '현대상선', 'HMM'],
    'KT': ['Korea Telecom', 'KT', '케이티'],
    '코웨이': ['Coway', '코웨이'],
    '팬오션': ['Pan Ocean', '팬오션'],
    '삼성SDI': ['삼성에스디아이', 'Samsung SDI', '삼성SDI'],
    '신한지주': ['Shinhan Financial Group', '신한금융지주', '신한금융', '신한'],
    '한국가스공사': ['한국가스공사', 'KOGAS', '가스공사'],
    '농심': ['Nongshim', '농심'],
    '영원무역': ['Youngone', '영원무역'],
    '금호타이어': ['Kumho Tire', '금호타이어'],
    '이마트': ['E-mart', '이마트'],
    '삼성생명': ['삼성생명보험', '삼성생명금융서비스', '삼성생명', '삼생'],
    '고려아연': ['Korea Zinc', '고려아연', 'KZ'],
    'S-Oil': ['에쓰오일', 'S-Oil', '에스오일', 'S-오일'],
    '엔씨소프트': ['엔씨소프트', 'NCsoft', 'NC소프트', 'NC'],
    '카카오페이': ['KakaoPay', '카카오페이', '카페이']
}

In [ ]:


# Alphabet list
alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def normalize_text(text):
    """텍스트를 유니코드 NFC 형식으로 정규화, 비문자형 데이터는 빈 문자열로 변환"""
    if isinstance(text, str):
        return unicodedata.normalize('NFC', text)
    else:
        return ''  # NaN 또는 비문자형 데이터를 빈 문자열로 처리

def clean_company_name(name):
    """기업명 정리 및 표준화 (기업명 매핑 딕셔너리 적용)"""
    cleaned_name = normalize_text(name).strip()

# Find a name that matches in the dictionary
    for standard_name, variations in company_name_mapping.items():
        if cleaned_name in variations:
            return standard_name

# If there is nothing matched, the original name is returned
    return cleaned_name

def preprocess_content(content):
    """본문 내용에서 기업명 대체 작업 (조사가 붙어도 매핑되도록 보완)"""
    for original_name, variations in sorted(company_name_mapping.items(), key=lambda x: -len(x[0])):
        content = re.sub(rf'({"|".join(map(re.escape, variations))})([가-힣]+)?',
                         lambda m: original_name + (m.group(2) if m.group(2) else ''),
                         content)
    return content

def generate_anonymized_name(used_names, alphabet):
    """사용되지 않은 알파벳을 사용해 익명화된 이름을 생성"""
    while True:
        anonymized_name = random.choice(alphabet) + '사'
        if anonymized_name not in used_names:
            used_names.add(anonymized_name)
            return anonymized_name

def anonymize_article(content, companies, used_names, alphabet):
    """기업명 익명화 (길이순으로 긴 이름부터 처리)"""
    company_to_anonymous = {}
    companies = sorted(companies, key=lambda x: -len(x))

    for company in companies:
        normalized_company = clean_company_name(company)
        if normalized_company not in company_to_anonymous:
            company_to_anonymous[normalized_company] = generate_anonymized_name(used_names, alphabet)

        try:
            content = re.sub(rf'({re.escape(company)})([가-힣]+)?',
                             lambda m: company_to_anonymous[normalized_company] + (m.group(2) if m.group(2) else ''),
                             content)
        except re.error as e:
            print(f"Regex error with company: {company}, Error: {str(e)}")

    return content, company_to_anonymous

def anonymize_company_names(df, all_companies, start_row=0):
    """메인 익명화 함수 (중단된 배치부터 이어서 처리 가능)"""
    alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

    tqdm_bar = tqdm(total=len(df), desc="진행상황", mininterval=1, initial=start_row)  # 중단된 곳에서 시작

    for i, row in df.iloc[start_row:].iterrows():  # 중단된 지점부터 처리
        used_names = set()  # 사용된 알파벳 집합
        content = row['content']

        if pd.isna(content):
            print(f"Warning: Row {i} has NaN content, skipping this row.")
            continue  # NaN 값을 가진 행은 건너뛰기

        content = normalize_text(content)
        content = preprocess_content(content)

        original_company_name = row['company']

        if pd.isna(original_company_name):
            print(f"Warning: Row {i} has NaN company name, skipping this row.")
            continue  # NaN 값을 가진 행은 건너뛰기

        original_company_name = clean_company_name(original_company_name)

        companies_in_article = [original_company_name]
        for company in all_companies:
            try:
                if re.search(rf'{re.escape(company)}', content):
                    companies_in_article.append(company)
            except re.error as e:
                print(f"Regex error with company: {company}, Error: {str(e)}")

        anonymized_content, company_to_anonymous = anonymize_article(content, companies_in_article, used_names, alphabet)

# .loc safe value assignment
        df.loc[i, 'content'] = anonymized_content
        df.loc[i, '기업명'] = company_to_anonymous.get(original_company_name, original_company_name)

# Output progress every 500th
        if i % 500 == 0:
            tqdm.write(f"처리 중인 행 번호: {i}")
            tqdm.write(f"탐지대상: {companies_in_article}")
            tqdm.write(f"처리대상: {company_to_anonymous}")
            tqdm.write(f"익명화중: {row['content']}")
            tqdm.write(f"익명화됨: {anonymized_content}")

        tqdm_bar.update(1)  # tqdm 진행 상태 업데이트

    tqdm_bar.close()
    return df

def check_completed_batches(output_dir, total_batches):
    """
    이미 처리된 배치 파일을 확인하여 진행 상태를 반환합니다.
    """
    completed_batches = []
    for i in range(total_batches):
        output_file = os.path.join(output_dir, f'A0-60_anonymized_batch_{i}.csv')
        if os.path.exists(output_file):
            df = pd.read_csv(output_file)
            completed_batches.append((i, len(df)))  # 배치 번호와 처리된 행 수를 저장
    return completed_batches

def process_batch(batch_file, start_row, all_companies, output_dir, batch_num):
    """
    주어진 배치 파일을 처리하고, 결과를 저장합니다.
    중단된 지점부터 다시 처리할 수 있도록 `start_row`부터 처리합니다.
    """
    df_batch = pd.read_csv(batch_file)

    anonymized_df = anonymize_company_names(df_batch, all_companies, start_row=start_row)

#Save full data including parts already processed
    output_file = os.path.join(output_dir, f'A0-60_anonymized_batch_{batch_num}.csv')
    anonymized_df.to_csv(output_file, index=False)
    print(f'Batch {batch_num} 처리 완료: {start_row}행부터 처리됨.')

def process_batches(batch_files, all_companies, output_dir):
    """
    전체 배치 파일을 순차적으로 처리합니다.
    이미 처리된 배치 파일이 있으면 그 이후부터 이어서 작업을 진행합니다.
    """
    total_batches = len(batch_files)

# Check the already handled batch
    completed_batches = check_completed_batches(output_dir, total_batches)
    completed_batch_nums = [batch[0] for batch in completed_batches]

    for batch_num in tqdm(range(total_batches), desc="배치 처리"):
        batch_file = batch_files[batch_num]

# Skip the already complete arrangement
        if batch_num in completed_batch_nums:
            print(f"Batch {batch_num} 이미 완료됨. 건너뜁니다.")
            continue

        print(f"Batch {batch_num} 처리 시작.")
        process_batch(batch_file, 0, all_companies, output_dir, batch_num)


In [ ]:
# Batch file list (original file name)
batch_files = [
    '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_batch/A0-60_batch_0.csv',
    '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_batch/A0-60_batch_1.csv',
    '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_batch/A0-60_batch_2.csv',
    '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_batch/A0-60_batch_3.csv'
]

# Directory to save after processing
output_dir = '/content/drive/MyDrive/Kwargs/020. 전처리/A0-60_anonymized_batch'

#Batch processing process execution
process_batches(batch_files, all_companies, output_dir)

배치 처리:   0%|          | 0/4 [00:00<?, ?it/s]

Batch 0 처리 시작.







진행상황:   1%|▏         | 145/9999 [06:57<23:45,  6.91it/s]




진행상황:   1%|▏         | 145/9999 [06:57<23:45,  6.91it/s]




진행상황:   1%|▏         | 145/9999 [06:57<23:45,  6.91it/s]




진행상황:   1%|▏         | 145/9999 [06:57<23:45,  6.91it/s]




진행상황:   1%|▏         | 145/9999 [06:57<23:45,  6.91it/s]

진행상황:   0%|          | 0/9999 [00:00<?, ?it/s]

처리 중인 행 번호: 0
탐지대상: ['삼성생명', '삼성전자', '신한지주', '한화생명', '한화', '삼성']
처리대상: {'삼성생명': 'X사', '삼성전자': 'B사', '신한지주': 'I사', '한화생명': 'T사', '한화': 'M사'}
익명화중: - 금감원에 ''즉시연금'' 민원 몰려- 70%가 B사·M사·교보생명에 집중- 소멸시효 연장 계약자 접수 몰려- 당국, 보험사에 과소지급액 재조사그래픽이데일리 이동훈 기자이데일리 즉시연금 보험금을 덜 받았다며 금융당국에 분쟁 조정을 신청하거나 소비자 단체의 공동소송에 참여한 보험 계약자가 2000명에 육박하는 것으로 나타났다. 특히 이중 빅3 보험사(B사·M사·교보생명)에 70%가량이 집중된 것으로 알려졌다. 지난해 생명보험 업계를 달군 즉시연금 과소 지급액 사태의 파문이 일파만파로 재확산될 가능성을 배제할 수 없는 셈이다. 여기에 금융감독 당국의 보험 담당 임원에 이른바 ‘보험사 저승사자’가 내정된 것으로 알려진데다 올해 본격 부활한 종합 검사의 화살까지 보험사를 겨누면서 보험업계는 좌불안석이다. 즉시연금 과소지급액 환급 2000명 신청금융감독원은 작년 9월부터 인터넷 홈페이지 등을 통해 즉시연금 분쟁 조정 일괄 신청 접수를 받은 결과, 현재까지 1700여 명이 신청했다고 14일 밝혔다. 이와 별개로 민간단체인 금융소비자연맹이 지난해 9월부터 두 차례에 걸쳐 즉시연금을 판매한 B사생명 등 15개 생명보험사를 상대로 덜 받은 보험금을 돌려받겠다며 모집한 공동소송 원고단에도 200명이 넘는 소비자가 몰렸다. 금융당국은 신청 건별로 분류 작업을 통해 실제 구제 대상(현재 1500여 명 추정)을 선별할 예정이다. 논란의 중심에 놓인 즉시연금(만기 환급형 또는 상속 만기형)은 최초 가입 때 보험료를 한꺼번에 내면 보험사가 매달 이자(연금)를 가입자에게 지급하고 계약 만기 때 처음에 납부한 보험료 전액을 돌려주는 상품이다. 앞서 지난해 3월 금감원은 B사생명이 즉시연금 상품 계약자에게 이자를 덜 줬다는 금융분쟁조



진행상황:   0%|          | 7/9999 [00:01<26:23,  6.31it/s]

진행상황:   0%|          | 15/9999 [00:02<23:54,  6.96it/s]

진행상황:   0%|          | 23/9999 [00:03<23:27,  7.09it/s]

진행상황:   0%|          | 31/9999 [00:04<23:16,  7.14it/s]

진행상황:   0%|          | 39/9999 [00:05<23:22,  7.10it/s]

진행상황:   0%|          | 47/9999 [00:06<23:20,  7.10it/s]

진행상황:   1%|          | 55/9999 [00:07<22:59,  7.21it/s]

진행상황:   1%|          | 63/9999 [00:08<22:51,  7.25it/s]

진행상황:   1%|          | 71/9999 [00:09<22:35,  7.32it/s]

진행상황:   1%|          | 79/9999 [00:10<22:26,  7.37it/s]

진행상황:   1%|          | 87/9999 [00:12<22:22,  7.38it/s]



진행상황:   1%|          | 95/9999 [00:13<22:32,  7.32it/s]

진행상황:   1%|          | 103/9999 [00:14<22:20,  7.38it/s]

진행상황:   1%|          | 111/9999 [00:15<22:24,  7.36it/s]

진행상황:   1%|          | 119/9999 [00:16<22:32,  7.31it/s]

진행상황:   1%|▏         | 127/9999 [00:17<22:40,  7.25it/s]

진행상황:   1%|▏         | 135/9999 [00:18<22:51,  7.19it/s]

진행상황:   1%|▏         | 143/9999 [00:19<22:32,  7.29it/s]

배치 처리:   0%|          | 0/4 [02:37<?, ?it/s]


KeyboardInterrupt: 